# Project 2: 8 Puzzle Solver Part One -- Iterative Deepening
## Author: Jared Rand

### Purpose: 
Write a search algorithm that that finds a solution for an 8 puzzle (A square puzzle with three rows of smaller squares, two rows with 3 and one with 2 and an empty space. The purpose is to put the squares in the correct order only by sliding the squares next to the gap.)

<img src="img/sol05.gif">

# Introduction:
In this project we were meant to find a solution to a given 8 puzzle using iterative deepening. This project is very similar to my Missionaries and Cannibals solution, as they both use essentially the same tree file. 

# Discussion Of Solution:
This project wasn't a huge amount of work, as I was able to take my solution to the previous project and just apply it to this one. It's almost the same kind of solution too, I have a tree that makes a node for every possible legal move. 

The class for the nodes is a bit more extensive than the one I made for the previous projects.

In [ ]:
class Node:
    def __init__(self, name, data):
        self.name = name
        self.children = []
        self.parent = []
        self.data = data
        self.end = False

        self.position = self.getPos()  # defaults
        self.left = self.getLeft()
        self.right = self.getRight()
        self.up = self.getUp()
        self.down = self.getDown()

    def new_child(self, name):
        self.children.append(name)

    def getPos(self):
        aa = str(np.where(self.data == 0))
        res = []
        i = 0
        for each in aa:
            if each == '[':
                res.append(int(aa[i + 1]))
            i += 1
        return res

    def swap(self, element):
        mypos = self.getPos()
        aa = str(np.where(self.data == element))
        res = []
        i = 0
        for each in aa:
            if each == '[':
                res.append(int(aa[i + 1]))
            i += 1
        self.data[mypos[0]][mypos[1]] = element
        self.data[res[0]][res[1]] = 0

    def moveUp(self):
        self.swap(self.up)
        self.refreshNeighbors()

    def moveDown(self):
        self.swap(self.down)
        self.refreshNeighbors()

    def moveLeft(self):
        self.swap(self.left)
        self.refreshNeighbors()

    def moveRight(self):
        self.swap(self.right)
        self.refreshNeighbors()

    def getUp(self):
        a = self.position
        return self.data[a[0] - 1][a[1]]

    def getDown(self):
        a = self.position
        return self.data[a[0] + 1][a[1]]

    def getLeft(self):
        a = self.position
        return self.data[a[0]][a[1] - 1]

    def getRight(self):
        a = self.position
        return self.data[a[0]][a[1] + 1]

    def refreshNeighbors(self):
        self.position = self.getPos()
        self.up = self.getUp()
        self.down = self.getDown()
        self.left = self.getLeft()
        self.right = self.getRight()


I do all of the swapping and such inside of the class itself, which makes the tree class look slightly neater than it did before. I also have each node keep track of:

1. The entire board
2. Its position on the board.
3. Its neighboring numbers.
4. Its parents and its children

I'm hoping this isn't what makes it way more sluggish than the previous project, but I will discuss that later. 

To make sure it doesn't make any bad or unnecessary moves, I have 2 ways of checking. The first is that the board is surrounded by -1s, so if the blank space tries to swap with anything outside the bounds, it ends that node. Like this:

-1,-1,-1,-1,-1

-1, 1, 2, 3, -1

-1, 3, 4, 5, -1

-1, 6, 7, 8, -1

-1,-1,-1,-1,-1

The second check is here:


In [ ]:
 if noodle.name != "ENDED":
            parr = parent
            while parr.name != "0-0(ROOT)":
                if np.array_equal(noodle.data, parr.data):
                    noodle.end = True
                    noodle.name = "ENDED"
                    self.n -= 1
                parr = parr.parent
            parent.new_child(noodle)  # Adds new node as child to its parent node
            noodle.parent = parent  # Adds parent to new node

This runs through each nodes parents all the way back to the root node, and if it's at the same step as one of its parents then it ends the node.

This is also likely a spot where it slows the program down, as it keeps having to go through all the parent's data for each node. If I have time to improve on this, I might try using a dictionary to hold the previous values, but delete all the previous nodes. I'm not sure if it would work but it's just a thought. 


# Testing Design and Expected Results

For the output of my program, when it gets to around 15 layers deep, it starts to really slow down. It has to go through each node and make around 4 more, so the number of nodes is almost quadrupling every level. The number of nodes gets super high and slows the program down significantly. If it takes less than 10 moves to solve the puzzle, then it should find it within a few seconds. 

I think one of the main reasons why this program creates so many nodes is because, unlike the missionaries and cannibals, it has so many moves it can make that. Even if it the moves are technically legal, it might not even be in the correct direction, so it ends up creating a lot of unnecessary paths that bog up the program. 

# Actual Results:

The results should look something like this:

<img src="img/Capture.jpg">

The above shows the all the moves that are needed to make in order to solve the puzzle. This specific one takes 6 moves. 

# Discussion Of Results:

Overall this program could run much better, but I'm happy that it works in general. I'm sure it could solve the boards that take a higher number of moves if I let it take the time to do so, but I don't want to set my computer on fire. 

# Link To Azure Notebook:

https://notebooks.azure.com/randja/projects/8-puzzle-part-one

# Code Below Runs Everything:

## NOTE: I made a "Demo" file that's essentially the same as the regular file, except it only goes down 10 layers (So it doesn't bog down the jupyter notebook). If takes over 10 moves to solve the 8 puzzle, it starts a new one until it finds one with less than 10 moves. This is why the output might look a little strange.

## Jupyter Notebooks makes it act weird if you try to run it multiple times, so restart the kernel or download the files if you want to see it run again. 


In [ ]:
from myTreeDEMO import *
import numpy as np
import random


def apply_action(board, action):

    deltas = np.array([[-1, 0, 1, 0], [0, 1, 0, -1]])
    action_2_index = {'up': 0, 'right': 1, 'down': 2, 'left': 3}
    posx, posy = np.where(np.isin(board, [0]))
    (x, y) = (posx[0], posy[0])
    (new_x, new_y) = (x + deltas[0, action_2_index[action]], y + deltas[1, action_2_index[action]])

    try:
        el = board[new_x, new_y]
        board[x, y] = el
        board[new_x, new_y] = 0
       # print(new_x, new_y)
       # print(board)
    except IndexError:
        pass
    return board


def mess_up(board, actions, moves):

    for iter in range(0, moves):
        board = apply_action(board, actions[random.randint(0, 3)])
    pass

### Impossible Board Checking ###
### Method Found Here: ###
### https://learnbycoding.wordpress.com/2014/12/25/how-to-check-if-an-instance-of-8-puzzle-is-solvable/ ###
def getCount(arr):
    invcount = 0
    i = 0
    while i < 9-1:
        j = i+1
        while j < 9:
            if arr[j] and arr[i] and arr[i] > arr[j]:
                invcount += 1
            j += 1
        i += 1
    return invcount


def makeFlatter(puz):
    bbb = []
    j = 0
    k = 0
    for i in range(9):
        bbb.append(puz[j][k])
        k += 1
        if k > 2:
            k = 0
            j += 1
    return bbb


def checkSolvable(puzzle, goal):
    flatpuz = makeFlatter(puzzle)
    invCount = getCount(flatpuz)
    if invCount % 2 == 0 or not np.array_equal(puzzle, goal):
        return True
    else:
        actions = ["up", "right", "down", "left"]
        mess_up(puzzle, actions, 12)
        return checkSolvable(puzzle)


def do():
    goal = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 0]])
    messupboard = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 0]])
    board = np.array([[-1, -1, -1, -1, -1], [-1, 1, 2, 3, -1], [-1, 4, 5, 6, -1], [-1, 7, 8, 0, -1], [-1, -1, -1, -1, -1]])
    actions = ["up", "right", "down", "left"]
    mess_up(messupboard, actions, 12)
    checkSolvable(messupboard, goal)
    for i in range(3):
        board[1][i+1] = messupboard[0][i]
        board[2][i+1] = messupboard[1][i]
        board[3][i+1] = messupboard[2][i]

    print("START BOARD:\n{}\n\n".format(messupboard))
    goal = np.array([[-1, -1, -1, -1, -1], [-1, 1, 2, 3, -1], [-1, 4, 5, 6, -1], [-1, 7, 8, 0, -1], [-1, -1, -1, -1, -1]])
    newtree = MyTree(board, goal, possibles=actions)
    return find_solution(newtree, goal)


def main():
    x = do()
    while x == 0:
        do()


if __name__ == "__main__":
    main()


START BOARD:
[[1 2 3]
 [0 6 4]
 [7 5 8]]


GENERATED LEVEL 1 (3 NODES)
GENERATED LEVEL 2 (8 NODES)
GENERATED LEVEL 3 (16 NODES)
GENERATED LEVEL 4 (24 NODES)
GENERATED LEVEL 5 (48 NODES)
GENERATED LEVEL 6 (72 NODES)
GENERATED LEVEL 7 (144 NODES)
GENERATED LEVEL 8 (216 NODES)
GENERATED LEVEL 9 (432 NODES)
GENERATED LEVEL 10 (648 NODES)
START BOARD:
[[1 2 3]
 [4 8 0]
 [7 6 5]]


GENERATED LEVEL 1 (6 NODES)
GENERATED LEVEL 2 (16 NODES)
GENERATED LEVEL 3 (32 NODES)
GENERATED LEVEL 4 (48 NODES)
GENERATED LEVEL 5 (96 NODES)

:::::::::::::::::::::::::::::::::
::: SOLUTION FOUND IN 0.24s :::::
::: NODE 5-75    ::::::::::::::::
:::::::::::::::::::::::::::::::::

PATH:

0-0(ROOT)
///////////
// 1|2|3 //
// 4|8|0 //
// 7|6|5 //
///////////

1-4
///////////
// 1|2|3 //
// 4|8|5 //
// 7|6|0 //
///////////

2-11
///////////
// 1|2|3 //
// 4|8|5 //
// 7|0|6 //
///////////

3-22
///////////
// 1|2|3 //
// 4|0|5 //
// 7|8|6 //
///////////

4-37
///////////
// 1|2|3 //
// 4|5|0 //
// 7|8|6 //
///////////